In [2]:
!pip install Keras==2.2.4

In [11]:
!pip install tensorflow==1.14.0

  Using cached https://files.pythonhosted.org/packages/de/f0/96fb2e0412ae9692dbf400e5b04432885f677ad6241c088ccc5fe7724d69/tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl


In [12]:
!pip install tensorflow-gpu==1.14.0 --ignore-installed

  Using cached https://files.pythonhosted.org/packages/76/04/43153bfdfcf6c9a4c38ecdb971ca9a75b9a791bb69a764d652c359aca504/tensorflow_gpu-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d/absl_py-0.9.0-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f3/97/f74da84d4db8cfe95f9b6aa2469be79af1873fec1adb80405105ed99a0a8/grpcio-1.28.1-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/57/02/5432412c162989260fab61fa65e0a490c1872739eb91a659896e4d554b26/protobuf-3.11.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8c/23/848298cccf8e40f5bbb59009b32848a4c38f4e7f3364297ab3c3e2e2cd14/wheel-0.34.2-py2.py3-none-any.whl
  Using cached https://files

In [1]:
import pandas as pd
import numpy as np

from numpy.random import seed
from tensorflow import set_random_seed
import tensorflow as tf
import random as rn
import os
from nltk import tokenize 
import nltk
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#2. Get the file
if os.name=='posix':
    
    # GOOGLE COLAB SETUP
    # Load the Drive helper and mount
    from google.colab import drive
    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
    codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/AOBDL_DL'
    cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models_formspring'
    models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models_formspring'
    
else:
    data_path          = '../data'
    codes_path         = '../AOBDL_DL'
    cv_models_path     = '../cv_models_formspring'
    models_path        = '../models_formspring'

In [0]:
import sys
sys.path.append(f'{codes_path}')

In [4]:
if tf.test.is_gpu_available():
  import models
else:
  import models_no_gpu as models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [0]:
#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/formspring_cleaned_no_punkt.csv') 
train.text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
max_features = 7000
maxlen       = 300
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150
threshold    = 0.4


#max_sen_len    = 100
#max_sent_amount = 4

seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
length = pd.Series(X).apply(lambda x: len(x.split()))

In [12]:
length.describe()

count    9342.000000
mean       22.160672
std        21.608183
min         2.000000
25%        11.000000
50%        17.000000
75%        27.000000
max       812.000000
dtype: float64

# LSTM

In [0]:
LSTM_threshold = 0.1

In [49]:
models.train_model(
    X=X, 
    y=y,
    mtype = 'LSTM',
    cv=True, 
    nfolds=5, 
    epochs=4, 
    cv_models_path=cv_models_path, 
    train=False, 
    rs=rs, 
    max_features=max_features, 
    maxlen=maxlen,
    threshold=LSTM_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
7473/7473 [==============================] - 4s 536us/step - loss: 0.2822 - acc: 0.9229
Epoch 2/4
7473/7473 [==============================] - 3s 428us/step - loss: 0.2340 - acc: 0.9356
Epoch 3/4
7473/7473 [==============================] - 3s 430us/step - loss: 0.1976 - acc: 0.9372
Epoch 4/4
1869/1869 [==============================] - 0s 253us/step
 0.1 fold 0 precision 0.592 recall 0.372 fscore 0.457
fold 0 average precision 0.507
fold 0 roc auc 0.862
 fold 1
Fitting
Epoch 1/4
7473/7473 [==============================] - 4s 476us/step - loss: 0.2795 - acc: 0.9196
Epoch 2/4
7473/7473 [==============================] - 3s 431us/step - loss: 0.2355 - acc: 0.9356
Epoch 3/4
7473/7473 [==============================] - 3s 432us/step - loss: 0.2006 - acc: 0.9370
Epoch 4/4
1869/1869 [==============================] - 0s 162us/step
 0.1 fold 1 precision 0.273 recall 0.785 fscore 0.405
fold 1 average precision 0.558
fold 1 roc auc 0.884
 fold 2
Fitting
Epoch 1/4
7474

In [50]:
models.train_model(X=X, 
                    y=y, 
                    X_test=X_test, 
                    y_test=y_test, 
                    mtype = 'LSTM', 
                    cv=False, 
                    epochs=4, 
                    cv_models_path=models_path, 
                    train=False, 
                    rs=rs,
                    max_features=max_features, 
                    maxlen=maxlen,
                    threshold=LSTM_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 5s 482us/step - loss: 0.2714 - acc: 0.9283
Epoch 2/4
9342/9342 [==============================] - 4s 427us/step - loss: 0.2256 - acc: 0.9358
Epoch 3/4
9342/9342 [==============================] - 4s 429us/step - loss: 0.1658 - acc: 0.9437
Epoch 4/4
3817/3817 [==============================] - 1s 148us/step
_________________________________
PR-C is 0.522
_________________________________

_________________________________
ROC AUC is 0.872
_________________________________
_________________________________
FScore is 0.339
_________________________________



# GRU

In [0]:
GRU_threshold = 0.3

In [52]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'GRU', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=GRU_threshold)

 fold 0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fitting
Epoch 1/4
7473/7473 [==============================] - 3s 446us/step - loss: 0.3141 - acc: 0.9209
Epoch 2/4
7473/7473 [==============================] - 3s 398us/step - loss: 0.2029 - acc: 0.9366
Epoch 3/4
7473/7473 [==============================] - 3s 392us/step - loss: 0.1566 - acc: 0.9441
Epoch 4/4
1869/1869 [==============================] - 0s 169us/step
 0.3 fold 0 precision 0.446 recall 0.446 fscore 0.446
fold 0 average precision 0.497
fold 0 roc auc 0.877
 fold 1
Fitting
Epoch 1/4
7473/7473 [==============================] - 3s 449us/step - loss: 0.3181 - acc: 0.9197
Epoch 2/4
7473/7473 [==============================] - 3s 399us/step - loss: 0.2011 - acc: 0.9359
Epoch 3/4
7473/7473 [==============================] - 3s 392us/step - loss: 0.1567 - acc: 0.9454
Epoch 4/4
1869/1869 [==============================] - 0s 156us/step
 0.3 fold 1 precision 0.681 recall 0.405 fscore 0.508
fold 1 average precision 0.565
fold 1 roc auc 0.89
 fold 2
Fitting
Epoch 1/4
7474/7474 [==

In [53]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'GRU', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=GRU_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 4s 428us/step - loss: 0.3040 - acc: 0.9237
Epoch 2/4
9342/9342 [==============================] - 4s 393us/step - loss: 0.1927 - acc: 0.9368
Epoch 3/4
9342/9342 [==============================] - 4s 394us/step - loss: 0.1473 - acc: 0.9497
Epoch 4/4
3817/3817 [==============================] - 0s 129us/step
_________________________________
PR-C is 0.537
_________________________________

_________________________________
ROC AUC is 0.867
_________________________________
_________________________________
FScore is 0.516
_________________________________



# CNN

In [0]:
cnn_threshold = 0.3

In [55]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'CNN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=cnn_threshold)

 fold 0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fitting
Epoch 1/4
7473/7473 [==============================] - 2s 330us/step - loss: 0.2930 - acc: 0.9062
Epoch 2/4
7473/7473 [==============================] - 2s 247us/step - loss: 0.2131 - acc: 0.9358
Epoch 3/4
7473/7473 [==============================] - 2s 262us/step - loss: 0.1545 - acc: 0.9471
Epoch 4/4
1869/1869 [==============================] - 0s 103us/step
 0.3 fold 0 precision 0.746 recall 0.388 fscore 0.511
fold 0 average precision 0.553
fold 0 roc auc 0.861
 fold 1
Fitting
Epoch 1/4
7473/7473 [==============================] - 2s 289us/step - loss: 0.2817 - acc: 0.9356
Epoch 2/4
7473/7473 [==============================] - 2s 247us/step - loss: 0.2131 - acc: 0.9356
Epoch 3/4
7473/7473 [==============================] - 2s 247us/step - loss: 0.1572 - acc: 0.9438
Epoch 4/4
1869/1869 [==============================] - 0s 79us/step
 0.3 fold 1 precision 0.602 recall 0.463 fscore 0.523
fold 1 average precision 0.564
fold 1 roc auc 0.915
 fold 2
Fitting
Epoch 1/4
7474/7474 [==

In [56]:
models.train_model(X=X, 
                  y=y, 
                  X_test=X_test, 
                  y_test=y_test, 
                  mtype = 'CNN', 
                  cv=False, 
                  epochs=4, 
                  cv_models_path=models_path, 
                  train=False, 
                  rs=rs,
                  max_features=max_features, 
                  maxlen=maxlen,
                  threshold=cnn_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 3s 285us/step - loss: 0.2806 - acc: 0.9118
Epoch 2/4
9342/9342 [==============================] - 2s 248us/step - loss: 0.1963 - acc: 0.9377
Epoch 3/4
9342/9342 [==============================] - 2s 246us/step - loss: 0.1443 - acc: 0.9524
Epoch 4/4
3817/3817 [==============================] - 0s 79us/step
_________________________________
PR-C is 0.586
_________________________________

_________________________________
ROC AUC is 0.906
_________________________________
_________________________________
FScore is 0.552
_________________________________



# BGRU

In [0]:
BGRU_threshold = 0.2

In [58]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BGRU_threshold)

 fold 0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fitting
Epoch 1/4
7473/7473 [==============================] - 6s 828us/step - loss: 0.2934 - acc: 0.9338
Epoch 2/4
7473/7473 [==============================] - 6s 764us/step - loss: 0.2182 - acc: 0.9356
Epoch 3/4
7473/7473 [==============================] - 6s 759us/step - loss: 0.1596 - acc: 0.9425
Epoch 4/4
1869/1869 [==============================] - 1s 298us/step
 0.2 fold 0 precision 0.509 recall 0.463 fscore 0.485
fold 0 average precision 0.513
fold 0 roc auc 0.878
 fold 1
Fitting
Epoch 1/4
7473/7473 [==============================] - 6s 814us/step - loss: 0.3007 - acc: 0.9283
Epoch 2/4
7473/7473 [==============================] - 6s 756us/step - loss: 0.2084 - acc: 0.9356
Epoch 3/4
7473/7473 [==============================] - 6s 756us/step - loss: 0.1645 - acc: 0.9423
Epoch 4/4
1869/1869 [==============================] - 1s 284us/step
 0.2 fold 1 precision 0.675 recall 0.463 fscore 0.549
fold 1 average precision 0.571
fold 1 roc auc 0.889
 fold 2
Fitting
Epoch 1/4
7474/7474 [=

In [59]:
models.train_model(X=X, 
                y=y, 
                X_test=X_test, 
                y_test=y_test, 
                mtype = 'BGRU', 
                cv=False, 
                epochs=4, 
                cv_models_path=models_path, 
                train=False,
                rs=rs,
                max_features=max_features, 
                maxlen=maxlen,
                threshold=BGRU_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 8s 828us/step - loss: 0.2862 - acc: 0.9142
Epoch 2/4
9342/9342 [==============================] - 7s 753us/step - loss: 0.1976 - acc: 0.9364
Epoch 3/4
9342/9342 [==============================] - 7s 754us/step - loss: 0.1466 - acc: 0.9483
Epoch 4/4
3817/3817 [==============================] - 1s 257us/step
_________________________________
PR-C is 0.514
_________________________________

_________________________________
ROC AUC is 0.86
_________________________________
_________________________________
FScore is 0.405
_________________________________



# BLSTM

In [0]:
BLSTM_threshold = 0.1

In [65]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BLSTM', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BLSTM_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
1869/1869 [==============================] - 1s 310us/step
 0.1 fold 0 precision 0.618 recall 0.347 fscore 0.444
fold 0 average precision 0.493
fold 0 roc auc 0.857
 fold 1
Fitting
1869/1869 [==============================] - 1s 309us/step
 0.1 fold 1 precision 0.479 recall 0.653 fscore 0.552
fold 1 average precision 0.556
fold 1 roc auc 0.89
 fold 2
Fitting
1868/1868 [==============================] - 1s 313us/step
 0.1 fold 2 precision 0.337 recall 0.7 fscore 0.455
fold 2 average precision 0.536
fold 2 roc auc 0.889
 fold 3
Fitting
1868/1868 [==============================] - 1s 310us/step
 0.1 fold 3 precision 0.566 recall 0.5 fscore 0.531
fold 3 average precision 0.522
fold 3 roc auc 0.836
 fold 4
Fitting
1868/1868 [==============================] - 1s 302us/step
 0.1 fold 4 precision 0.625 recall 0.333 fscore 0.435
fold 4 average precision 0.486
fold 4 roc auc 0.868
PR-C 0.518
ROC AUC 0.868
FScore 0.484
PR-C std 0.026
ROC AUC std 0.021
FScore std 0.048


In [66]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BLSTM', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BLSTM_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 8s 899us/step - loss: 0.2767 - acc: 0.9301
Epoch 2/4
9342/9342 [==============================] - 8s 823us/step - loss: 0.2177 - acc: 0.9361
Epoch 3/4
9342/9342 [==============================] - 8s 821us/step - loss: 0.1535 - acc: 0.9487
Epoch 4/4
3817/3817 [==============================] - 1s 281us/step
_________________________________
PR-C is 0.558
_________________________________

_________________________________
ROC AUC is 0.871
_________________________________
_________________________________
FScore is 0.414
_________________________________



# BGRU + Max Pooling

In [0]:
BGRU_max_threshold = 0.3

In [73]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_max', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BGRU_max_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
1869/1869 [==============================] - 1s 294us/step
 0.3 fold 0 precision 0.634 recall 0.43 fscore 0.512
fold 0 average precision 0.521
fold 0 roc auc 0.868
 fold 1
Fitting
1869/1869 [==============================] - 1s 292us/step
 0.3 fold 1 precision 0.645 recall 0.405 fscore 0.497
fold 1 average precision 0.525
fold 1 roc auc 0.892
 fold 2
Fitting
1868/1868 [==============================] - 1s 298us/step
 0.3 fold 2 precision 0.659 recall 0.242 fscore 0.354
fold 2 average precision 0.52
fold 2 roc auc 0.892
 fold 3
Fitting
1868/1868 [==============================] - 1s 295us/step
 0.3 fold 3 precision 0.451 recall 0.658 fscore 0.536
fold 3 average precision 0.591
fold 3 roc auc 0.885
 fold 4
Fitting
1868/1868 [==============================] - 1s 297us/step
 0.3 fold 4 precision 0.765 recall 0.325 fscore 0.456
fold 4 average precision 0.542
fold 4 roc auc 0.87
PR-C 0.54
ROC AUC 0.881
FScore 0.471
PR-C std 0.027
ROC AUC std 0.011
FScore std 0.064


In [74]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_max', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BGRU_max_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 8s 879us/step - loss: 0.2886 - acc: 0.9166
Epoch 2/4
9342/9342 [==============================] - 8s 817us/step - loss: 0.2400 - acc: 0.9356
Epoch 3/4
9342/9342 [==============================] - 8s 819us/step - loss: 0.2084 - acc: 0.9357
Epoch 4/4
3817/3817 [==============================] - 1s 277us/step
_________________________________
PR-C is 0.549
_________________________________

_________________________________
ROC AUC is 0.883
_________________________________
_________________________________
FScore is 0.511
_________________________________



# BGRU + attention

In [0]:
BGRU_att_len = 50
BGRU_att_threshold = 0.3

In [114]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_att', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_att_len,
                   threshold=BGRU_att_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
1869/1869 [==============================] - 0s 137us/step
 0.3 fold 0 precision 0.508 recall 0.504 fscore 0.506
fold 0 average precision 0.523
fold 0 roc auc 0.849
 fold 1
Fitting
1869/1869 [==============================] - 0s 130us/step
 0.3 fold 1 precision 0.622 recall 0.421 fscore 0.502
fold 1 average precision 0.524
fold 1 roc auc 0.865
 fold 2
Fitting
1868/1868 [==============================] - 0s 126us/step
 0.3 fold 2 precision 0.524 recall 0.633 fscore 0.574
fold 2 average precision 0.567
fold 2 roc auc 0.885
 fold 3
Fitting
1868/1868 [==============================] - 0s 131us/step
 0.3 fold 3 precision 0.447 recall 0.592 fscore 0.509
fold 3 average precision 0.541
fold 3 roc auc 0.875
 fold 4
Fitting
1868/1868 [==============================] - 0s 131us/step
 0.3 fold 4 precision 0.909 recall 0.167 fscore 0.282
fold 4 average precision 0.467
fold 4 roc auc 0.856
PR-C 0.524
ROC AUC 0.866
FScore 0.475
PR-C std 0.033
ROC AUC std 0.013
FScore std 0.1


In [115]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_att', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_att_len,
                   threshold=BGRU_att_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 3s 276us/step - loss: 0.2920 - acc: 0.9143
Epoch 2/4
9342/9342 [==============================] - 2s 195us/step - loss: 0.2319 - acc: 0.9356
Epoch 3/4
9342/9342 [==============================] - 2s 195us/step - loss: 0.2001 - acc: 0.9358
Epoch 4/4
3817/3817 [==============================] - 0s 87us/step
_________________________________
PR-C is 0.52
_________________________________

_________________________________
ROC AUC is 0.874
_________________________________
_________________________________
FScore is 0.51
_________________________________



# BGRU + Average Pooling

In [0]:
BGRU_avg_len = 50
BGRU_avg_threshold = 0.2

In [125]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_avg', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False,
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_avg_len,
                   threshold=BGRU_avg_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
1869/1869 [==============================] - 0s 102us/step
 0.2 fold 0 precision 0.671 recall 0.405 fscore 0.505
fold 0 average precision 0.527
fold 0 roc auc 0.875
 fold 1
Fitting
1869/1869 [==============================] - 0s 104us/step
 0.2 fold 1 precision 0.724 recall 0.347 fscore 0.469
fold 1 average precision 0.556
fold 1 roc auc 0.894
 fold 2
Fitting
1868/1868 [==============================] - 0s 111us/step
 0.2 fold 2 precision 0.618 recall 0.392 fscore 0.48
fold 2 average precision 0.527
fold 2 roc auc 0.891
 fold 3
Fitting
1868/1868 [==============================] - 0s 101us/step
 0.2 fold 3 precision 0.481 recall 0.525 fscore 0.502
fold 3 average precision 0.563
fold 3 roc auc 0.864
 fold 4
Fitting
1868/1868 [==============================] - 0s 102us/step
 0.2 fold 4 precision 0.527 recall 0.408 fscore 0.46
fold 4 average precision 0.508
fold 4 roc auc 0.884
PR-C 0.536
ROC AUC 0.882
FScore 0.483
PR-C std 0.02
ROC AUC std 0.011
FScore std 0.018


In [126]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_avg', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_avg_len,
                   threshold=BGRU_avg_threshold)

Fitting
Epoch 1/4
9342/9342 [==============================] - 2s 230us/step - loss: 0.2880 - acc: 0.9313
Epoch 2/4
9342/9342 [==============================] - 1s 157us/step - loss: 0.2260 - acc: 0.9357
Epoch 3/4
9342/9342 [==============================] - 1s 157us/step - loss: 0.1686 - acc: 0.9443
Epoch 4/4
3817/3817 [==============================] - 0s 74us/step
_________________________________
PR-C is 0.544
_________________________________

_________________________________
ROC AUC is 0.874
_________________________________
_________________________________
FScore is 0.324
_________________________________



# psHAN

In [0]:
psHAN_max_sen_len = 20
psHAN_max_sent_amount = 8
psHAN_threshold = 0.1

In [143]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'psHAN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=psHAN_max_sent_amount*psHAN_max_sen_len,
                   max_sen_len=psHAN_max_sen_len,
                   max_sent_amount=psHAN_max_sent_amount,
                   threshold=psHAN_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
1869/1869 [==============================] - 0s 242us/step
 0.1 fold 0 precision 0.417 recall 0.537 fscore 0.469
fold 0 average precision 0.516
fold 0 roc auc 0.876
 fold 1
Fitting
1869/1869 [==============================] - 0s 224us/step
 0.1 fold 1 precision 0.469 recall 0.678 fscore 0.554
fold 1 average precision 0.587
fold 1 roc auc 0.877
 fold 2
Fitting
1868/1868 [==============================] - 0s 216us/step
 0.1 fold 2 precision 0.453 recall 0.725 fscore 0.558
fold 2 average precision 0.589
fold 2 roc auc 0.889
 fold 3
Fitting
1868/1868 [==============================] - 0s 249us/step
 0.1 fold 3 precision 0.41 recall 0.667 fscore 0.508
fold 3 average precision 0.583
fold 3 roc auc 0.883
 fold 4
Fitting
1868/1868 [==============================] - 0s 218us/step
 0.1 fold 4 precision 0.604 recall 0.533 fscore 0.566
fold 4 average precision 0.574
fold 4 roc auc 0.883
PR-C 0.57
ROC AUC 0.882
FScore 0.531
PR-C std 0.027
ROC AUC std 0.005
FScore std 0.037


In [153]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'psHAN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=psHAN_max_sent_amount*psHAN_max_sen_len,
                   max_sen_len=psHAN_max_sen_len,
                   max_sent_amount=psHAN_max_sent_amount,
                   threshold=psHAN_threshold)

Fitting
3817/3817 [==============================] - 1s 221us/step
_________________________________
PR-C is 0.596
_________________________________

_________________________________
ROC AUC is 0.879
_________________________________
_________________________________
FScore is 0.576
_________________________________



# HAN

In [0]:
#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/formspring_cleaned_sent_punkt.csv') 
train.text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
HAN_max_sen_len = 20
HAN_max_sent_amount = 8
HAN_threshold = 0.2

In [147]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'HAN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=HAN_max_sen_len*HAN_max_sent_amount,
                   max_sen_len=HAN_max_sen_len,
                   max_sent_amount=HAN_max_sent_amount,
                   threshold=HAN_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
0
0
Fitting
1869/1869 [==============================] - 1s 306us/step
 0.2 fold 0 precision 0.362 recall 0.562 fscore 0.44
fold 0 average precision 0.534
fold 0 roc auc 0.855
 fold 1
0
0
Fitting
1869/1869 [==============================] - 0s 219us/step
 0.2 fold 1 precision 0.5 recall 0.43 fscore 0.462
fold 1 average precision 0.526
fold 1 roc auc 0.894
 fold 2
0
0
Fitting
1868/1868 [==============================] - 0s 209us/step
 0.2 fold 2 precision 0.625 recall 0.542 fscore 0.58
fold 2 average precision 0.581
fold 2 roc auc 0.904
 fold 3
0
0
Fitting
1868/1868 [==============================] - 0s 200us/step
 0.2 fold 3 precision 0.725 recall 0.483 fscore 0.58
fold 3 average precision 0.621
fold 3 roc auc 0.9
 fold 4
0
0
Fitting
1868/1868 [==============================] - 0s 216us/step
 0.2 fold 4 precision 0.496 recall 0.475 fscore 0.485
fold 4 average precision 0.45
fold 4 roc auc 0.871
PR-C 0.542
ROC AUC 0.885
FScore 0.51
PR-C std 0.058
ROC AUC std 0.019
FScore std 0.0

In [136]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'HAN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=HAN_max_sen_len*HAN_max_sent_amount,
                   max_sen_len=HAN_max_sen_len,
                   max_sent_amount=HAN_max_sent_amount,
                   threshold=HAN_threshold)

0
0
Fitting
Epoch 1/4
9342/9342 [==============================] - 6s 593us/step - loss: 0.2675 - acc: 0.9356
Epoch 2/4
9342/9342 [==============================] - 4s 459us/step - loss: 0.2426 - acc: 0.9356
Epoch 3/4
9342/9342 [==============================] - 4s 461us/step - loss: 0.1826 - acc: 0.9442
Epoch 4/4
3817/3817 [==============================] - 1s 178us/step
_________________________________
PR-C is 0.579
_________________________________

_________________________________
ROC AUC is 0.899
_________________________________
_________________________________
FScore is 0.554
_________________________________

